# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",   # this can be set as always to involve human input
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure! Here's a joke for you:

Why did the golfer bring two pairs of pants?

In case he got a hole in one!


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure, which joke would you like me to repeat?


* Note: the ConversableAgent did not remeber the previous joke it said. A different approach is needed to to remeber conversations.

## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing, Joe! So I recently started a new diet where I only eat alphabet soup. The funny thing is, I'm now feeling much more "well-read"!

--------------------------------------------------------------------------------
joe (to cathy):

That's great, Cathy! I'm just hoping all those letters don't start forming words in your stomach.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, good one, Joe! Well, if that happens, at least I'll have some very literate digestion!

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': 'Sure thing, Joe! So I recently started a new diet where I only '
             "eat alphabet soup. The funny thing is, I'm now feeling much more "
             '"well-read"!',
  'role': 'user'},
 {'content': "That's great, Cathy! I'm just hoping all those letters don't "
             'start forming words in your stomach.',
  'role': 'assistant'},
 {'content': "Haha, good one, Joe! Well, if that happens, at least I'll have "
             'some very literate digestion!',
  'role': 'user'}]


In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 85,
                                                             'cost': 0.00023999999999999998,
                                                             'prompt_tokens': 225,
                                                             'total_tokens': 310},
                                      'total_cost': 0.00023999999999999998},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 85,
                                                             'cost': 0.00023999999999999998,
                                                             'prompt_tokens': 225,
                                                             'total_tokens': 310},
                                      'total_cost': 0.00023999999999999998}}


In [9]:
pprint.pprint(chat_result.summary)

("Haha, good one, Joe! Well, if that happens, at least I'll have some very "
 'literate digestion!')


## Get a better summary of the conversation

In [10]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing, Joe! So I recently started a new diet where I only eat alphabet soup. The funny thing is, I'm now feeling much more "well-read"!

--------------------------------------------------------------------------------
joe (to cathy):

That's great, Cathy! I'm just hoping all those letters don't start forming words in your stomach.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, good one, Joe! Well, if that happens, at least I'll have some very literate digestion!

--------------------------------------------------------------------------------


In [11]:
pprint.pprint(chat_result.summary)

('Joe and Cathy shared jokes about a funny diet and the possibility of letters '
 "forming words in Cathy's stomach.")


## Chat Termination

Chat can be terminated using a termination conditions.

In [12]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [13]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, are you ready for some laughs? So, I painted my laptop black, hoping it would run faster... Now it doesn't work. I think it caught a virus!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! It sounds like your laptop needs some tech support more than a paint job. Maybe try giving it some antivirus instead!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, you're right, Joe! My laptop needs a good dose of antivirus, not a makeover. Maybe I should stick to comedy and leave the tech stuff to the experts!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that might be for the best, Cathy! Stick to making people laugh and leave the com

In [14]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that would be about your laptop catching a virus after you painted it black to make it run faster. Classic!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that was a fun one! I gotta go now, Joe. Have a great day and keep spreading those laughs!

--------------------------------------------------------------------------------
